# Keras and TensorFlow.js Interoperability


## Introduction
With the release of TensorFlow.js earlier in 2018 (https://js.tensorflow.org/), it is now possible to train and run Machine Learning models directly in a web browser and NodeJS-based applications. TensorFlow.js implements much of TensorFlow's proven API and thus opens up interesting possibilities for enriching web applications with Machine Learning capabilities.

## Demo: Train offline, predict online
For many real-world ML problems which require very deep, recurrent, or convoluted neural networks, training of such models, while feasible, wouldn't be very practical to do in a browser. This is especially true for mobile applications.

A far more common use case, however, would be to use a pre-trained model to predictions. In the example of a neural network, training the model entails establishing the weights, which is typically orders of magnitudes more expensive in terms of CPU compared to just applying the trained weights to make predictions, based on new data.

## Part 1: Offline training

For this toy example, we'll use the classical Iris flower dataset. The goal here is to predict the type of flower based on four different measurements in cm taken from 150 flowers. The dataset is build into the `sklearn` package.

### Data loading and preparation

In [4]:
from sklearn import datasets
from sklearn import preprocessing
from keras.utils import to_categorical


# Load and transform the data.
iris = datasets.load_iris()
iris_target_onehot = to_categorical(iris.target)
X = preprocessing.scale(iris.data)
Y = iris_target_onehot

### Defining the model

For this multi-class classification problem, we'll be setting up a simple feed-forward neural network:

In [6]:
def build_iris_model(num_hidden_layers = 1, 
                     x_dim = 4, 
                     num_neurons = 64, 
                     my_optimizer = "adam"):
    """
    This function returns a feed-forward neural network, as
    defined in the parameters.
    """
    model = Sequential()
    model.add(Dense(num_neurons, 
                 input_dim = x_dim, 
                 activation = "relu"))
    
    # Add specified number of hidden layers.
    for i in range(num_hidden_layers):        
        model.add(Dense(num_neurons, 
                        activation = "relu"))
    
    model.add(Dense(3, activation = "softmax"))
    
    # Compile the model according to the selected optimizer function.
    # Using categorical cross entropy because we are classifying.
    model.compile(loss = "categorical_crossentropy", 
               optimizer = my_optimizer, 
               metrics = ["accuracy"])
    return model

### Training the model

Next, we'll fit the model to the training data (60% of the data).

**Note**: The data splitting isn't really necessary in this example as I'm not doing any validation after.


In [10]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size = 0.4, 
                                                    random_state = 0)

# Train the model.
model = build_iris_model()
model.fit(X_train, 
          Y_train, 
          validation_split = 0.25, 
          epochs = 150, 
          batch_size = 10, 
          verbose = 0)

### Testing the model

We can now convince ourselves that the model predicts well on the training data, as expected:


In [16]:
import numpy as np

print("The first sample should yield class 1: " + str(model.predict_classes(np.reshape(X_train[0, :], (1, 4)))))
print("The second sample should yield class 0: " + str(model.predict_classes(np.reshape(X_train[50, :], (1, 4)))))

The first sample should yield class 1: [1]
The second sample should yield class 0: [0]


### Saving the model

So far so good, nothing really new. Now we need to save the model.

Out of the box, Keras supports the HDF5 model format. However, for TensorFlow.js we need to convert into TensorFlow's Layers format. This can be achieved with a utility function within the `tensorflowjs` package.

**Note** For the next line to work, the `tensorflowjs` package needs to be installed:

* Anaconda: `pip install tensorflowjs`
* Kaggle: Add a custom package.


In [20]:
import tensorflowjs as tfjs

TFJS_MODEL_PATH = "C:/Temp/irismodel-tfjs"
tfjs.converters.save_keras_model(model, TFJS_MODEL_PATH)

## Part 2: Using the model online